# Data-Import

In [13]:
import pandas as pd
import ast
import re

# Set the display options to show all columns and rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

# Data import Aisha
original_dataframe = pd.read_csv("recipes_w_search_terms.csv")

# data import katie
# original_dataframe = pd.read_csv("C:\\Users\\k.ritscher\\Desktop\\Bootcamp\\Final_Project\\archive\\recipes_w_search_terms.csv")

### Dropping Duplicates, Creating smaller DF

In [14]:
# drop duplicates
df = original_dataframe.copy()
df.drop_duplicates(
    subset=["name", "ingredients", "description"], keep="first", inplace=True
)

In [15]:
# see difference in the size of the df,
print(len(original_dataframe))
print(len(df))

494963
494777


In [25]:
# creating a smaller test-dataframe
test_df = df.iloc[30:59, :]

## Creating columns 
time to make, low-in-something, free-off-something, cuisine


### Create column for "basic_ingredients"

In [17]:
df["ingredients"].dtypes  # the datatype of ingredients seems to be an object

dtype('O')

In [19]:
[
    ingredient
    for ingredient in [
        "salt",
        "oil",
        "water",
    ]
    if ingredient
    in test_df["ingredients"][0]
    .strip("][]")
    .replace("'", "")
    .split(
        ", "
    )  # transform column to a list by removing squarebrackets and quotes and split by commas
]
# next: loop through ingredients again

['salt', 'water']

In [ ]:
[
    ingredient_list.strip("][").replace("'", "").split(", ")
    for ingredient_list in test_df["ingredients"].to_list()
]

In [32]:
# New column for Basic ingredients

# Create a loop to check for ingredients in ingredients column
ingredient_check = [
    ingredient
    for ingredient in [
        "salt",
        "oil",
        "water",
        "pepper",
        "vegetable oil",
        "garlic",
        "olive oil",
        "white wine vinegar",
        "table salt",
        "black pepper",
        "pepper and salt",
        "salt and pepper",
        "italian seasoning",
        "flour",
        "allspice",
        "cinnamon",
        "sugar",
        "ground cumin",
        "cumin",
        "red bell pepper",
        "cayenne pepper",
        "nutmeg",
        "baking powder",
        "all purpose flour",
        "white vinegar",
        "ice",
        "cardamom",
        "kosher salt",
        "oregano",
    ]
    if any(
        ingredient in row
        for row in test_df["ingredients"]
        .str.strip("[]")
        .str.replace("'", "")
        .str.split(", ")
    )
]

# Replace " ' " and separate by ,
processed_ingredients = [
    ingredient_list.replace("'", "").split(", ")
    for ingredient_list in test_df["ingredients"].to_list()
]

# Create new column with basic ingredients
basic_ingredients_list = [
    "salt",
    "oil",
    "water",
    "pepper",
    "vegetable oil",
    "garlic",
    "olive oil",
    "white wine vinegar",
    "table salt",
    "black pepper",
    "pepper and salt",
    "salt and pepper",
    "italian seasoning",
    "flour",
    "allspice",
    "cinnamon",
    "sugar",
    "ground cumin",
    "cumin",
    "red bell pepper",
    "cayenne pepper",
    "nutmeg",
    "baking powder",
    "all purpose flour",
    "white vinegar",
    "ice",
    "cardamom",
    "kosher salt",
    "oregano",
]
test_df["basic_ingredients"] = [
    [ingredient for ingredient in basic_ingredients_list if ingredient in row]
    for row in test_df["ingredients"]
    .str.strip("[]")
    .str.replace("'", "")
    .str.split(", ")
]

C:\Users\k.ritscher\AppData\Local\Temp\ipykernel_7176\4256298537.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["basic_ingredients"] = [


In [33]:
def calculate_missing_values(row):
    set_a = set(row["ingredients"])
    set_b = set(row["basic_ingredients"])
    missing_values = list(set_a - set_b)
    return missing_values


# Apply the function to create the new 'time-to-make' column
test_df["non_basic_ingredients"] = df["ingredients"].apply(
    lambda x: calculate_missing_values(ast.literal_eval(x)) if pd.notnull(x) else None
)
test_df.head(1)

TypeError: list indices must be integers or slices, not str

In [48]:
test_df["non_basic_ingredients"] = ~(
    test_df["ingredients"].isin(test_df["basic_ingredients"])
)

In [49]:
df = pd.DataFrame({"country": ["US", "UK", "Germany", "China"]})
countries_to_keep = ["UK", "China"]

# pseudo-code:
df[df["country"] not in countries_to_keep]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [45]:
test_df

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,basic_ingredients,non_basic_ingredients
30,232111,"Double Pork, Double Cheese Burgers","Food and Wine Magazine, June /07 edition. ""These burgers pair well with the crisp, coriander-inflected Lagunitas Brewing Company’s Sonoma Farmhouse Saison, modeled after the widely distributed Belgian beer Saison Dupont—also a great match."" Chef Sang Yoon will almost always choose ale over Riesling and this is one of his beer-friendly recipes.","['bacon', 'ground pork', 'garlic clove', 'thyme', 'kosher salt', 'fresh ground pepper', 'red onions', 'olive oil', 'hamburger buns', 'unsalted butter', 'camembert cheese', 'gorgonzola', 'arugula', 'tomatoes']","[""1/2 cup finely chopped bacon"",""1 1/2 lbs ground pork"",""1 large garlic clove, minced "",""2 teaspoons finely chopped thyme"",""1 teaspoon kosher salt"",""1/2 teaspoon fresh ground pepper"",""4 slices red onions (1/2-inch thick)"","" olive oil, for drizzling "",""4 hamburger buns or 4 ciabatta rolls"",""2 tablespoons unsalted butter, melted "",""6 ounces camembert cheese, cut into eight slices (1/3-inch-thick)"",""2 ounces gorgonzola, cut into four slices (1/3-inch-thick)"","" arugula"",""4 tomatoes, slices ""]",1 (435 g),4,"['In a skillet, cook the bacon over moderate heat, stirring until cooked through, about 3 minutes.', 'Transfer the bacon to paper towels to drain and let cool.', 'In a large bowl, mix the bacon with the ground pork, garlic, thyme, salt and pepper.', 'Shape into four 1-inch-thick patties.', 'Light a grill.', 'Drizzle the onion slices with olive oil and grill over moderately high heat until lightly charred, about 2 minutes per side.', 'Transfer to a platter.', 'Brush the cut sides of the buns with the melted butter and grill, cut sides down, until toasted, about 1 minute.', 'Turn and grill for 30 seconds longer.', 'Transfer the buns to the platter.', 'Grill the burgers until charred outside and cooked through, about 5 minutes per side.', 'Arrange 2 Camembert slices and a Gorgonzola slice on each burger and cook until the cheese is melted, 1 minute.', 'On the bottom halves of the buns, layer the arugula, tomato, burgers and grilled onion.', 'Close the burgers and serve.', '*The burgers can be refrigerated overnight. Bring to room temperature before grilling.']","['bacon', '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'main-dish', 'eggs-dairy', 'pork', 'easy', 'holiday-event', 'cheese', 'independence-day', 'meat', 'pork-loins', 'superbowl']","{'pork', 'dinner'}","[olive oil, kosher salt]",True
31,251759,Brain Cookies With Blood Glaze,A great halloween cookie,"['unsalted butter', 'sugar', 'all-purpose flour', 'baking soda', 'salt', 'eggs', 'walnuts', 'vanilla', 'red food coloring', 'blue food coloring', 'icing sugar', 'red food coloring']","["""",""1 cup unsalted butter, softened "",""1 cup sugar"",""3 cups all-purpose flour"",""1/2 teaspoon baking soda"",""1/2 teaspoon salt"",""2 large eggs"",""1/2 cup very finely chopped walnuts or 1/2 cup pecans"",""1 teaspoon vanilla"",""5 drops red food coloring"",""9 drops blue food coloring"","""",""2 cups icing sugar"",""35 drops red food coloring""]",1 (52 g),24,"['Preheat the oven to 350 degrees F.', 'Line 2 large baking sheets with parchment paper and set aside.', 'In a large bowl, cream together the butter and sugar.', 'Into a separate bowl, sift together the flour, baking soda, and salt.', 'Alternating with the eggs, add the flour mixture to the butter mixture, beating well after the each addition.', 'Fold in the nuts, vanilla, and red and blue food coloring, being careful not to overmix the dough.', 'Place the dough in batches in a potato grinder and push the dough out onto the prepared baking sheets in long tubes of dough.', 'With your fingers, loosely pat and arrange the dough strands into clumps resembling brains, pushing to form 2 hemispheres and shaping into a walnut-like s

### Create column for "time to make"

In [ ]:
# Function to extract 'time-to-make' value based on 'minutes' or 'hours'
def extract_time_to_make(tags_list):
    time_to_make_values = []
    for tag in tags_list:
        if "minutes" in tag.lower() or "hours" in tag.lower():
            time_to_make_values.append(tag.strip())
    return time_to_make_values if time_to_make_values else None


# Apply the function to create the new 'time-to-make' column
test_df["time-to-make"] = df["tags"].apply(
    lambda x: extract_time_to_make(ast.literal_eval(x)) if pd.notnull(x) else None
)
test_df.head(1)

### Create a column for low in something

In [ ]:
import re


# Function to extract 'low-in-something' value based on 'low'
def extract_low_in_something(low_values_list):
    low_in_something_values = []
    for tag in low_values_list:
        # Check if the tag has 'low' as a whole word
        if re.search(r"\blow\b", tag.lower()):
            low_in_something_values.append(tag.strip())
    return low_in_something_values if low_in_something_values else None


# Apply the function to create the new 'low-in-something' column
test_df["low-in-something"] = test_df["tags"].apply(
    lambda x: extract_low_in_something(ast.literal_eval(x)) if pd.notnull(x) else None
)
test_df.head(1)

/var/folders/jl/tcgfk4mx3gg3jtw3rcbk08pw0000gp/T/ipykernel_2993/2069912245.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["low-in-something"] = test_df["tags"].apply(


,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,basic_ingredients,main_ingredients,time-to-make,low-in-something
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}","[salt, water]","[grits, garlic]",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]"


### Add a column for "free-of-something"

In [ ]:
import re


# Function to extract 'free-of-something' value based on 'free'
def extract_free_of_something(free_values_list):
    free_of_something_values = []
    for tag in free_values_list:
        # Check if the tag has 'free' as a whole word
        if re.search(r"\bfree\b", tag.lower()):
            free_of_something_values.append(tag.strip())
    return free_of_something_values if free_of_something_values else None


# Apply the function to create the new 'free-of-something' column
test_df["free-of-something"] = test_df["tags"].apply(
    lambda x: extract_free_of_something(ast.literal_eval(x)) if pd.notnull(x) else None
)
test_df.head(1)

/var/folders/jl/tcgfk4mx3gg3jtw3rcbk08pw0000gp/T/ipykernel_2993/3955187298.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["free-of-something"] = test_df["tags"].apply(


,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,basic_ingredients,main_ingredients,time-to-make,low-in-something,free-of-something
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}","[salt, water]","[grits, garlic]",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]",None


### Add a column for cuisines 

In [ ]:
# List of predefined cuisines
cuisines = [
    "indian",
    "american",
    "lebanese",
    "italian",
    "chinese",
    "mexican",
    "japanese",
    "french",
    "german",
    "greek",
    "spanish",
    "thai",
    "korean",
    "vietnamese",
    "brazilian",
    "moroccan",
    "turkish",
    "russian",
    "swedish",
    "caribbean",
    "ethiopian",
    "peruvian",
    "malaysian",
    "australian",
    "south african",
    "portuguese",
    "cajun/creole",
    "argentine",
    "filipino",
    "middle-eastern",
    "north-american",
    "central-american",
    "south-american",
    "european",
    "eastern-european",
    "western-european",
    "southeast-asian",
    "south-asian",
    "middle-eastern",
    "north-african",
    "sub-saharan-african",
    "south-indian",
    "north-indian",
    "southern-american",
    "northern-american",
    "central-european",
    "northern-european",
    "southern-european",
    "eastern-asian",
    "western-asian",
    "northern-african",
    "southern-african",
    "east-african",
    "west-african",
    "eastern-european",
    "western-european",
    "southeast-asian",
    "south-asian",
    "east-asian",
    "central-asian",
    "western-asian",
    "middle-eastern",
    "south-east-asian",
    "north-east-asian",
    "south-central-asian",
    "north-central-asian",
    "south-west-asian",
    "north-west-asian",
    "south-east-european",
    "north-east-european",
    "south-central-european",
    "north-central-european",
    "south-west-european",
    "north-west-european",
    "south-east-african",
    "north-east-african",
    "south-central-african",
    "north-central-african",
    "south-west-african",
    "north-west-african",
    "southwest",
    "southwestern-us",
    "southwestern-american",
    "southwestern",
    "south-west-pacific",
]


# Function to extract 'cuisine' value
def extract_cuisines(tags_list):
    cuisine_list = [tag.strip() for tag in tags_list if tag.lower() in cuisines]
    return cuisine_list if cuisine_list else None


# Apply the function to create the new 'cuisine' column
test_df["cuisine"] = test_df["tags"].apply(
    lambda x: extract_cuisines(ast.literal_eval(x)) if pd.notnull(x) else None
)

test_df.head(1)

/var/folders/jl/tcgfk4mx3gg3jtw3rcbk08pw0000gp/T/ipykernel_2993/3480976722.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["cuisine"] = test_df["tags"].apply(


,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,basic_ingredients,main_ingredients,time-to-make,low-in-something,free-of-something,cuisine
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}","[salt, water]","[grits, garlic]",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]",None,None


### Add a column for Main Ingredients

In [ ]:
import ast
import re


# Function to apply to each row
def find_main_ingredients(row):
    # Recipe name (converted to lowercase)
    recipe_name = row["name"].lower()

    # Convert the string representation of the list to an actual list of strings
    ingredients_list = ast.literal_eval(row["ingredients"])

    # Extract words from ingredients and convert to lowercase
    ingredients_lower = [
        word.lower().strip()
        for ingredient in ingredients_list
        for word in re.findall(r"\b[\w\s-]+\b", ingredient)
    ]

    # Find ingredients mentioned in the recipe name
    main_ingredients = []

    for ingredient in ingredients_lower:
        if ingredient in recipe_name or any(
            part in recipe_name for part in ingredient.split()
        ):
            main_ingredients.append(ingredient)

    return main_ingredients


# Apply the function to each row in 'test_df'
test_df["main_ingredients"] = test_df.apply(find_main_ingredients, axis=1)

# Display the resulting DataFrame
test_df

/var/folders/jl/tcgfk4mx3gg3jtw3rcbk08pw0000gp/T/ipykernel_2993/321099713.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["main_ingredients"] = test_df.apply(find_main_ingredients, axis=1)


,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,basic_ingredients,main_ingredients,time-to-make,low-in-something,free-of-something,cuisine
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}","[salt, water]","[grits, cheddar cheese, garlic]",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]",None,None
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are in the mood for jambalaya. My kids loved it! From Cook's Illustrated ""Quick Recipe"" Cookbook.","['onion', 'red bell pepper', 'garlic cloves', 'large shrimp', 'salt', 'hot pepper sauce', 'vegetable oil', 'andouille sausage', 'long grain rice', 'bay leaves', 'diced tomatoes', 'clam juice', 'fresh parsley']","[""1 medium onion, chopped coarse "",""1 medium red bell pepper, chopped coarse "",""5 medium garlic cloves, chopped coarse "",""1 lb extra large shrimp, shelled and deveined "","" salt"","" hot pepper sauce"",""1 tablespoon vegetable oil"",""3/4 lb andouille sausage, halved lengthwise and then cut into 1/4 inch slices "",""1 1/2 cups long grain rice"",""4 bay leaves"",""1 (14 ounce) can diced tomatoes, briefly drained "",""2 (8 ounce) bottles clam juice"",""1/4 cup fresh parsley, chopped ""]",1 (366 g),4,"['In a food processor, pulse the onion, red pepper and garlic until chopped very fine, ten to twelve 1-second pulses, scraping down the sides of the bowl as necessary; set aside.', 'Season the shrimp with salt and hot pepper sauce to taste.', 'Heat the oil in a large Dutch oven over medium high heat until shimmering. Add the shrimp in a single layer and cook, without stirring, for 30 seconds. Using tongs, flip the shrimp and cook for another 30 seconds. Transfer the shrimp to a medium bowl and set aside.', 'Add the sausage to the pan and cook, stirring occasionally, until lightly browned, about 3 minutes. Using a slotted spoon, transfer the sausage to a second bowl.', 'Scrape the veggies from the food processor into the empty pot and cook, stirring frequently, until softened, about 3 minutes. Stir in the rice and continue to stir until the grains are coated with the fat, about 1 minute.', 'Add the bay leaves, tomatoes, clam juice, and 1 cup water and bring to a boil.', 'Stir in the sausage, cover, and reduce the heat to low.', 'Cook until the rice is tender, 17 to 20 minutes.', 'Off the heat, stir in the 

# Unused Code

In [ ]:
# def filter_basic_ingredients(ingredient_list):
#     basic_ingredients = [
#         "salt",
#         "oil",
#         "water",
#     ]

#     return [
#         ingredient
#         for ingredient in basic_ingredients
#         if ingredient.lower() in ingredient_list
#     ]

In [ ]:
# # Function to ensure that each cell is a list
# def ensure_list(cell):
#     if isinstance(cell, list):
#         return cell
#     else:
#         return [cell]


# # Apply the function to the "ingredients" column
# test_df["ingredients"] = test_df["ingredients"].apply(ensure_list)

# test_df.head(1)

In [ ]:
# filter_basic_ingredients(test_df["ingredients"])

In [ ]:
# import numpy as np

# def filter_basic_ingredients(ingredient_list):
#     return [
#         ingredient
#         for ingredient in ingredient_list
#         if ingredient.lower() in basic_ingredients
#     ]

#     return [
#         ingredient
#         for ingredient in ingredient_list
#         if ingredient.lower() in basic_ingredients
#     ]

# # test_df["filtered_ingredients"] = np.where(
# #     df["ingredients"].isin(basic_ingredients), "Yes", "No"
# # )

# test_df["filtered_ingredients"] = np.where(
#     test_df["ingredients"].isin(basic_ingredients),
#     (test_df["ingredients"].apply(filter_basic_ingredients())),
# )

In [ ]:
# def filter_basic_ingredients(ingredient_list):
#     basic_ingredients = [
#         "salt",
#         "oil",
#         "water",
#     ]  # Removed the extra double quote in 'salt'

#     return [
#         ingredient
#         for ingredient in ingredient_list
#         if ingredient.lower() in basic_ingredients
#     ]  # Use lower() to perform case-insensitive comparison


# test_df["filtered_ingredients"] = test_df[["ingredients"][0]].apply(
#     filter_basic_ingredients
# )

# # Display the modified DataFrame
# test_df.head(1)

In [ ]:
# def filter_basic_ingredients(ingredient_list):
#     basic_ingredients = ["salt", "oil", "water"]

#     filtered_list = []
#     for sublist in ingredient_list:
#         for ingredient in sublist:
#             if ingredient.lower() in basic_ingredients:
#                 filtered_list.append(ingredient.lower())

#     return filtered_list

In [ ]:
# test_df["filtered_ingredients"] = test_df["ingredients"].apply(
# filter_basic_ingredients()
# )

# test_df["filtered_ingredients"] = np.where(
#     df["ingredients"].isin(basic_ingredients), "Yes", "No"
# )

# test_df["filtered_ingredients"] = filter_basic_ingredients(test_df["ingredients"])


# Display the modified DataFrame
# test_df.head(1)

# look into  apply method, stack overflow
# otherwise, find an alternative (copy column, extract ingredients, send to yanish)

In [ ]:
# # New column for Basic ingredients

# # Create a loop to check for ingredients in ingredients column
# ingredient_check = [
#     ingredient
#     for ingredient in ["salt", "oil", "water", "pepper", "vegetable oil"]
#     if any(ingredient in row for row in test_df["ingredients"].str.strip("[]").str.replace("'", "").str.split(", "))
# ]

# # Replace " ' " and separate by ,
# processed_ingredients = [
#     ingredient_list.replace("'", "").split(", ")
#     for ingredient_list in test_df["ingredients"].to_list()
# ]

# # Create new column with basic ingredientsThird block
# basic_ingredients_list = ["salt", "oil", "water", "pepper", "vegetable oil"]
# test_df["basic_ingredients"] = [
#     [ingredient for ingredient in basic_ingredients_list if ingredient in row]
#     for row in test_df["ingredients"].str.strip("[]").str.replace("'", "").str.split(", ")
# ]

# test_df.head(2)

In [ ]:
# test_df["basic_ingredients"] = [
#     ingredient
#     for ingredient in [
#         "salt",
#         "oil",
#         "water",
#     ]
#     if ingredient
#     in test_df["ingredients"][1]
#     .strip("][]")
#     .replace("'", "")
#     .split(
#         ", "
#     )  # transform column to a list by removing squarebrackets and quotes and split by commas
# ]

# test_df.head(1)

In [ ]:
# # New column for Basic ingredients

# # Create a loop to check for ingredients in ingredients column
# ingredient_check = [
#     ingredient
#     for ingredient in ["salt", "oil", "water", "pepper", "vegetable oil"]
#     if any(ingredient in row for row in test_df["ingredients"].str.strip("[]").str.replace("'", "").str.split(", "))
# ]

# # Replace " ' " and separate by ,
# processed_ingredients = [
#     ingredient_list.replace("'", "").split(", ")
#     for ingredient_list in test_df["ingredients"].to_list()
# ]

# # Create new column with basic ingredientsThird block
# basic_ingredients_list = ["salt", "oil", "water", "pepper", "vegetable oil"]
# test_df["basic_ingredients"] = [
#     [ingredient for ingredient in basic_ingredients_list if ingredient in row]
#     for row in test_df["ingredients"].str.strip("[]").str.replace("'", "").str.split(", ")
# ]

# test_df.head(2)

In [ ]:
# [
#     ingredient_list.strip("][").replace("'", "").split(", ")
#     for ingredient_list in test_df["ingredients"].to_list()
# ]